Ticker selections
=================

In [1]:
# If you would like to refresh your data, please execute the bellow codes.

import pandas as pd
from datetime import datetime
from tqdm import tqdm

from mypo import Loader

DOWNLOAD = False

if DOWNLOAD:
    tickers = pd.read_csv("/app/docs/tutorial/tickers.csv")
    loader = Loader()
    for row in tqdm(tickers.to_dict('record')):
        loader.get(row['Ticker'], expense_ratio=row['ExpenseRatio'] * 0.01)
    loader.save('/app/docs/tutorial/all.bin')

In [2]:
loader = Loader.load('/app/docs/tutorial/all.bin')
loader = loader.since(datetime(2008, 1, 1))
market = loader.get_market()

In [3]:
summary = loader.summary()
summary

,established,names,total_assets,volume,expense_ratio
AGG,2003-09-29,iShares Core U.S. Aggregate Bond ETF,85938724864,6466483,0.0005
FPX,2006-05-24,First Trust US Equity Opportunities ETF,2084153472,120866,0.0057
FDN,2006-06-23,First Trust Dow Jones Internet Index Fund,10462989312,258733,0.0051
QCLN,2007-02-14,First Trust NASDAQ Clean Edge Green Energy Ind...,2947022336,533483,0.0060
FVD,2003-08-27,First Trust Value Line Dividend Index Fund,10207452160,1312757,0.0070
...,...,...,...,...,...
EEM,2003-04-14,iShares MSCI Emerging Markets ETF,29338421248,65868433,0.0070
DIA,1998-01-20,SPDR Dow Jones Industrial Average ETF Trust,25793642496,4995800,0.0016
DBC,2006-02-06,Invesco DB Commodity Index Tracking Fund,1891861760,2817116,0.0087
DBA,2007-01-05,Invesco DB Agriculture Fund,895486016,1276150,0.0091


In [15]:
from mypo import clustering_tickers, make_combinations
cluster = clustering_tickers(market, n=4)
combinations = make_combinations(cluster)

In [ ]:
from multiprocessing import Pool
from mypo.optimizer import MinimumVarianceOptimizer
import numpy as np

def proc(c):
    optimizer = MinimumVarianceOptimizer(span=50000)
    target_market = market.filter(list(c))
    optimizer.optimize(target_market, at=datetime.today())
    w = optimizer.get_weights()
    r = target_market.get_rate_of_change().to_numpy()
    q = np.dot(np.dot(w, np.cov(r.T)), w.T)
    r = np.dot(w, r.mean(axis=0))
    return target_market.get_tickers(), r, q


with Pool(6) as pool:
    imap = pool.imap(proc, combinations)
    result = list(tqdm(imap, total=len(combinations)))


df = pd.DataFrame({
    "c": [r[0] for r in result],
    "r": [r[1] for r in result],
    "q": [r[2] for r in result]
    })

  5%|▌         | 27706/535304 [07:51<2:33:14, 55.21it/s]

In [ ]:

df = pd.DataFrame({
    "c": [r[0] for r in result],
    "r": [r[1] for r in result],
    "q": [r[2] for r in result]
})
df['sharp ratio'] = df["r"] / df["q"]


print(df.sort_values("sharp ratio", ascending=False).head(10))
print(df.sort_values("r", ascending=False).head(10))
print(df.sort_values("q").head(10))



In [ ]:
import yfinance as yf

tickers = ['XLY', 'XLV', 'VHT', 'FVD', 'QQQ', 'VGT', 'XLK', 'XLF', 'VFH']
tickers += ['RWR', 'EWY', 'FXI', 'EWZ', 'XLU']
tickers += ['SHY', 'TLT', 'BBH', 'GLD', 'TIP', 'LQD']
for t in tickers:
    ticker = yf.Ticker(t)
    print(t, ticker.info['longName'])
    

In [ ]:
from mypo.optimizer import MinimumVarianceOptimizer
from mypo.rebalancer import MonthlyRebalancer
from mypo import Runner

target = market.filter(['XLY', 'RWR', 'SHY', 'BBH'])
print(target.get_rate_of_change())
optimizer = MinimumVarianceOptimizer()
runner = Runner(
    assets=[0.25, 0.25, 0.25, 0.25],
    rebalancer=MonthlyRebalancer(optimizer=optimizer, do_re_optimize=True),
    cash=0.5
)

runner.run(
    market=target,
    train_span=200,
    verbose=True
)
report = runner.report()
print(optimizer.get_weights())
print(yearly_total_return(report))
print(max_drawdown(report))
print(max_drawdown_span(report))
report.plot()